In [28]:
import pandas as pd
import re
from tqdm.notebook import tqdm
import tensorflow as tf

In [21]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollator, T5ForConditionalGeneration, T5TokenizerFast

from transformers import (
    T5ForConditionalGeneration, 
    T5Tokenizer, 
    EvalPrediction,
    DataCollator,
    Trainer,
    TrainingArguments)


### **Read reddit dataset**

In [10]:
!gdown --id 1OrtWVYzMEcCauJgP06kbaFuqRAL4UVEd

/usr/local/lib/python3.9/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1OrtWVYzMEcCauJgP06kbaFuqRAL4UVEd
To: /content/reddit_conversation.csv
100% 7.96M/7.96M [00:00<00:00, 98.3MB/s]


In [11]:
df = pd.read_csv('reddit_conversation.csv')

In [12]:
df.head()

,Unnamed: 0,0,1,2
0,0,What kind of phone(s) do you guys have?,I have a pixel. It's pretty great. Much better...,Does it really charge all the way in 15 min?
1,1,I have a pixel. It's pretty great. Much better...,Does it really charge all the way in 15 min?,"Pretty fast. I've never timed it, but it's und..."
2,2,Does it really charge all the way in 15 min?,"Pretty fast. I've never timed it, but it's und...","cool. I've been thinking of getting one, my ph..."
3,3,What kind of phone(s) do you guys have?,Samsung Galaxy J1. It's my first cell phone an...,What do you think of it? Anything you don't like?
4,4,Samsung Galaxy J1. It's my first cell phone an...,What do you think of it? Anything you don't like?,I love it. I can't think of anything I don't l...


In [15]:
Questions = list()
Answers = list()
for i in tqdm(range(len(df))):
  Q = df['0'][i]
  A = df['1'][i]

  Q = Q.lower()
  A = A.lower()

  Q = re.sub('[\/:;-_+@&!?$()<>.,@#%^&*"]',"",Q)
  A = re.sub('[\/:;-_+@&!?$()<>.,@#%^&*"]',"",A)

  A = "startseq "+A+" endseq"

  Questions.append(Q)
  Answers.append(A)

  0%|          | 0/56297 [00:00<?, ?it/s]

In [16]:
len(Questions)

56297

In [17]:
len(Answers)

56297

In [18]:
Questions[:10]

['what kind of phones do you guys have',
 "i have a pixel it's pretty great much better than what i had before ",
 'does it really charge all the way in 15 min',
 'what kind of phones do you guys have',
 "samsung galaxy j1 it's my first cell phone and i've had it for 7 months",
 "what do you think of it anything you don't like",
 'what kind of phones do you guys have',
 "lg optimus v i know it's old",
 'my friend told me to kill myself ',
 "don't kill yourself op"]

In [19]:
Answers[:10]

["startseq i have a pixel it's pretty great much better than what i had before  endseq",
 'startseq does it really charge all the way in 15 min endseq',
 "startseq pretty fast i've never timed it but it's under half an hour  endseq",
 "startseq samsung galaxy j1 it's my first cell phone and i've had it for 7 months endseq",
 "startseq what do you think of it anything you don't like endseq",
 "startseq i love it i can't think of anything i don't like about it endseq",
 "startseq lg optimus v i know it's old endseq",
 "startseq if it does it's job it's good enough endseq",
 "startseq don't kill yourself op endseq",
 "startseq i won't give them the satisfaction  endseq"]

In [23]:
data = pd.DataFrame({'question': Questions, 'answer': Answers})

In [24]:
data.shape

(56297, 2)

In [25]:
data.head()

,question,answer
0,what kind of phones do you guys have,startseq i have a pixel it's pretty great much...
1,i have a pixel it's pretty great much better t...,startseq does it really charge all the way in ...
2,does it really charge all the way in 15 min,startseq pretty fast i've never timed it but i...
3,what kind of phones do you guys have,startseq samsung galaxy j1 it's my first cell ...
4,samsung galaxy j1 it's my first cell phone and...,startseq what do you think of it anything you ...


In [81]:
from sklearn.model_selection import train_test_split

In [87]:
train, test = train_test_split(data, test_size=0.3, random_state=42)

In [88]:
print(train.shape)
print(test.shape)

(39407, 2)
(16890, 2)


**convert to huggingeface Dataset**

In [116]:
from datasets import Dataset, DatasetDict

In [124]:
train_dataset = Dataset.from_dict(train)
test_dataset = Dataset.from_dict(test)
raw_dataset = DatasetDict({'train': train_dataset, 'test': test_dataset})

In [125]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 39407
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 16890
    })
})

In [126]:
type(raw_dataset)

datasets.dataset_dict.DatasetDict

In [127]:
type(raw_dataset['train'])

datasets.arrow_dataset.Dataset

In [129]:
for sample in raw_dataset['train']:
  print(sample)
  break

{'question': 'my sunday is over\n\ne', 'answer': 'startseq rip endseq'}


In [133]:
print(raw_dataset['train']['question'][5])
print(raw_dataset['train']['answer'][5])

happy early birthday
startseq thank you happy bday for you today endseq


### **Preprocessing the data**

In [20]:
!pip install transformers
!pip install datasets
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 16.8 MB/s eta 0:00:0

In [22]:
checkpoint = "t5-base"
model = T5ForConditionalGeneration.from_pretrained(checkpoint)
tokenizer = T5TokenizerFast.from_pretrained(checkpoint)

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [134]:
max_input_length =  512
max_target_length = 64

# tokenize the examples
def convert_to_features(example_batch):

    input_encodings = tokenizer.batch_encode_plus(example_batch['question'], 
                                                  max_length=max_input_length, 
                                                  add_special_tokens=True,
                                                  truncation=True, 
                                                  pad_to_max_length=True)
    
    target_encodings = tokenizer.batch_encode_plus(example_batch['answer'], 
                                                   max_length=max_target_length, 
                                                   add_special_tokens=True,
                                                   truncation=True, pad_to_max_length=True)
                                                   
    encodings = {
        'input_ids': input_encodings['input_ids'], 
        'attention_mask': input_encodings['attention_mask'],
        'decoder_input_ids': target_encodings['input_ids'],
        'decoder_attention_mask': target_encodings['attention_mask']
    }

    return encodings


def add_eos_examples(example):
  example['question'] = example['question'] + " </s>"
  example['answer'] = example['answer'] + " </s>"
  return example

In [149]:
tokenized_dataset = raw_dataset.map(convert_to_features, batched=True)

Map:   0%|          | 0/39407 [00:00<?, ? examples/s]

Map:   0%|          | 0/16890 [00:00<?, ? examples/s]

In [150]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask'],
        num_rows: 39407
    })
    test: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask'],
        num_rows: 16890
    })
})

In [151]:
tokenized_dataset['train']['question'][5]

'happy early birthday'

In [152]:
tokenized_dataset = tokenized_dataset.remove_columns(
    ["question", "answer"]
)

train_dataset = tokenized_dataset["train"]
valid_dataset = tokenized_dataset["test"]

columns = ['input_ids', 'decoder_input_ids', 'attention_mask', 'decoder_attention_mask']
train_dataset.set_format(type='torch', columns=columns)
valid_dataset.set_format(type='torch', columns=columns)

In [153]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask'],
        num_rows: 39407
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask'],
        num_rows: 16890
    })
})

### **Fine-Tuning the t5 model**

In [161]:
from dataclasses import dataclass
from typing import List, Dict

In [162]:
@dataclass
class T2TDataCollator():
  def __call__(self, batch: List) -> Dict[str, torch.Tensor]:
    """
    Take a list of samples from a Dataset and collate them into a batch.
    Returns:
    A dictionary of tensors
    """
    
    input_ids = torch.stack([example['input_ids'] for example in batch])
    lm_labels = torch.stack([example['decoder_input_ids'] for example in batch])
    lm_labels[lm_labels[:, :] == 0] = -100 
    attention_mask = torch.stack([example['attention_mask'] for example in batch])
    decoder_attention_mask = torch.stack([example['decoder_attention_mask'] for example in batch])
    
    return {
        'input_ids': input_ids, 
        'attention_mask': attention_mask,
        'labels': lm_labels, 
        'decoder_attention_mask': decoder_attention_mask
    }

In [163]:
training_args = TrainingArguments(output_dir="./gdrive/My Drive/models", 
                                  per_device_train_batch_size=4, 
                                  per_device_eval_batch_size=4,
                                  gradient_accumulation_steps=16,
                                  learning_rate=1e-4, 
                                  num_train_epochs=7,
                                  logging_steps=100,
                                  run_name="end2end-chatbot-generative",
                                  evaluation_strategy="steps",
                                  save_steps=500,
                                  report_to="wandb",
                                  push_to_hub=True,
                                  push_to_hub_model_id="t5-end2end-chatbot-generative")

OSError: ignored